In [ ]:
import torch
import torchvision
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import h5py
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder
import timeit
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.datasets import fetch_openml
from sklearn.utils import check_random_state
import csv

In [ ]:
#DATASET splitted  
allSets = h5py.File('dataset/singleDigits_set.h5','r')
print(allSets.keys())
train_set = allSets['training_data']
train_labels = allSets['training_labels']
real_test_set = allSets['test_data']

train_set = np.array(train_set)
train_labels = np.array(train_labels)
real_test_set = np.array(real_test_set)
print("train_set.shape is " + str(train_set.shape))
print("real_test_set.shape is " + str(real_test_set.shape))



In [ ]:
# need to transform the dataset into a Tensor, how??? first transform images into array
train_set = np.asarray(train_set)
print(train_set.shape)
#print(train_set[0])
print(train_labels.shape)
#after split there are 167339 digits

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
def load_dataset():

    cv = KFold(n_splits=100000)
    count = 1
    for train_index, test_index in cv.split(train_set): # Run 5 times
        count += 1
        x, x_test, y, y_test = train_set[train_index], train_set[test_index], train_labels[train_index], train_labels[test_index]
        if(count==2):
            print(train_index)
            print(test_index)
            break
    print("x.shape = " + str(x.shape))
    print("y.shape = " + str(y.shape))
    
    y = y.reshape(1,-1)
    y_test = y_test.reshape(1, -1)

    y = np.squeeze(y)
    y = pd.Series(y)
    y = pd.get_dummies(y)
    y = np.array(y).T

    y_test = np.squeeze(y_test)
    y_test = pd.Series(y_test)
    y_test = pd.get_dummies(y_test)
    y_test = np.array(y_test).T  

    train_set_x_orig, train_set_y, test_set_x_orig, test_set_y = x,y,x_test,y_test
    return train_set_x_orig, train_set_y, test_set_x_orig, test_set_y
load_dataset()

In [ ]:
    train_set_x_orig, train_set_y, test_set_x_orig, test_set_y = load_dataset()
    train_set_x = train_set_x_orig.reshape((train_set_x_orig.shape[0], -1)).T / 255 
    test_set_x = test_set_x_orig.reshape((test_set_x_orig.shape[0], -1)).T / 255 
    
    real_test_set_dim = real_test_set.reshape((real_test_set.shape[0], -1)).T / 255 
    
    
    print(np.absolute(train_set_x.T[0].shape))
    print(np.absolute(train_set_y.T[4]))
    print(np.absolute(train_set_x.T[4,] - train_set_x.T[4,]).sum())

In [ ]:
#Read Index
test_set_index = h5py.File('dataset/test_set_index.h5','r')
print(test_set_index.keys())
set_index = test_set_index['test_set_index']


In [ ]:

def findPosition(index1): # index1 is the 0-13999
    result = []
    temp_position = -1
    for i in range(len(set_index)):
        if(set_index[i] == 0):
            temp_position += 1
        if(temp_position == index1):
            result.append(i)
            i += 1
            while(set_index[i] != 0):
                result.append(i)
                i += 1
            return result
        
def findConvertPosition(index3): # index1 is the 0-13999
    temp_position2 = -1
    for i in range(len(set_index)):
        if(set_index[i] == 0):
            temp_position2 += 1
        if(i == index3):
            return temp_position2
        
print(findPosition(12))
print(findConvertPosition(29628))

In [ ]:
# Check if there is the same image in training set
def printCorrectResult(index2):
    a1 = []
    a1 = findPosition(index2)
    for i in range(a1[0], a1[0]+len(a1)): # loop over Kaggle test
        print("----------------------")
        isDetected = False
        for j in range(0, len(train_set_y.T)): # loop over MNIST images
            if np.absolute((real_test_set_dim.T[i,] - train_set_x.T[j,])).sum() == 0: # If there is a same digit in training set.
                print("Found in train " + str(j))
                print(np.argmax(train_set_y.T[j], axis = 0))
                isDetected = True
        for j2 in range(0, len(real_test_set_dim.T)):
            if np.absolute((real_test_set_dim.T[i,] - real_test_set_dim.T[j2,])).sum() == 0 and i != j2:
                print("Found in test " + str(j2))
        if(isDetected == False):
             print("Not detected in image " + str(i))
        if(i % 10 == 9):
            print(str(i) + "isfinished")
        
printCorrectResult(7680)

In [ ]:
# Check all the answers
def checkCorrectResult(index2):
    c0 = 0
    answer = []
    a1 = []
    a1 = findPosition(index2)
    for i in range(a1[0], a1[0]+len(a1)): # loop over Kaggle test
        isDetected = False
        for j in range(0, 167444): # loop over MNIST images
            if np.absolute((real_test_set_dim.T[i,] - train_set_x.T[j,])).sum() == 0:
                isDetected = True
                answer.append(np.argmax(train_set_y.T[j], axis = 0))
                break
        if(isDetected == False):
            answer.append(-1)   
    while (len(a1) + c0 < 5):
        c0 += 1
        answer.append(1)
        answer.append(0)
    return answer

with open("generatedAnswers/finalResult.csv", 'rt', newline='') as f:
    data = csv.reader(f)
    c1 = 0
    for row in data:       
        if(c1 != 0):
            cp1 = checkCorrectResult(c1-1)
            cp2 = row[1]
            if(len(cp1) != len(cp2)):
                print("length problem in " +str(c1))
                c1 += 1
                continue
            for index in range (0, len(cp2)):
                if(int(cp1[index]) != -1 and (int(cp1[index]) != int(cp2[index]))):
                    print(cp1)
                    print(cp2)
                    print("error in " +str(c1))
        c1 += 1
        if(c1 % 50 == 0):
            print(str(c1) + "is done")